## Setup

From Kaggle-Competition "Dogs vs. Cats", a data set of 12500 dog images and 12500 cat images provided.
The goal is to build a classifier, that can predict if either a dog (=1) or a cat (=0) is in the image.
For the sake of running at low computational cost, we will only train the model on 1000 dog images and 1000 cat images.
Plus, we will only use 400 images of each class to evaluate the model.

## Importing data

In [1]:
#This is to import our images to keras
from keras.preprocessing.image import ImageDataGenerator

#specify, where
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
# set dimensions of images to 100x100
img_width, img_height = 100, 100
#total training images is 2000, namly 1000 dog images and 1000 cat images:
nb_train_samples = 2000
#total validation images is 800
#the validation images are only to evaluate the model and calculate the loss
nb_validation_samples = 800
#set epochs to 20: complete data set is run through 20 times
#too few epochs means bad accuracy
#too many epochs means overfitting and great computational cost
epochs = 20
#set batch_size to 20: gradients will be upgraded everytime after 20 images
batch_size = 20

#rescale train images: pixel values are between [0,255], rescale them
#between [0,1] for faster convergence
train_datagen = ImageDataGenerator( rescale = 1. / 255 )

#rescale test images accordingly
test_datagen = ImageDataGenerator( rescale = 1. / 255 )

#load training images from directory:
#subfolders named "dogs" and "cats" will be taken as labels
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

#validation data we will need to evaluate the loss and the model metrics after each epoch
#this data will NOT be trained
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')    

Using TensorFlow backend.
Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


## CNN model

In [2]:
#these imports are needed to build our Convolutional Neural Network (CNN)
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


#Sequential() will configure the model for training    
model = Sequential()

#First: Input image (100,100,3) for (width, height, channels)
#and connect input layer with 2D convolutional layer (this is needed for images);
#choose 32 filters and a kernel size of 3x3.
model.add(Conv2D(32, (3, 3), input_shape=(img_width,img_height,3)))
#choose rectified linear unit as activation function;
#most of the time this is industry standard
model.add(Activation('relu'))
#perform max pooling
#pool size of (2,2) to halve vertical and horizontal dimension of input
model.add(MaxPooling2D(pool_size=(2, 2)))

#perform this bundle of convolutions, activation and max pooling again
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#activation and max pooling again
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#flatten layers out
model.add(Flatten())
#create a dense layer of 32 neurons
model.add(Dense(32))
#perform relu
model.add(Activation('relu'))
#perform dropout of 0.5
#dropout will randomly swith neurons on and off
#with a 0.5 chance to minimize overfitting
model.add(Dropout(0.5))
#finish off with one output unit..
model.add(Dense(1))
#..the sigmoid function will act as a logistic function:
# a prediction as to 1 will predict its a dog
# and a prediction as to 0 will predict its a cat.
model.add(Activation('sigmoid'))

#lets configure the model for training:
#choose binary_crossentropy as loss-function:
#we have a binary classification problem
#for keras optimizer choose 'rmsprop',
#other optimizers e.g. 'adam' work well too
#activate metrics so training can be overlooked
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

## Training

In [3]:
#For large data sets like this one, use model.fit_generator
model.fit_generator(
    generator = train_generator,
    #total number of steps between two epochs:
    #set to (nb_train_samples / batch_size) and round off
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

#once training is done, save the whole model incl. architecture, weights etc.
model.save('CNN_model.h5')

#Go to predict.ipynb to use this trained CNN to predict dog vs. cat images

Epoch 1/20
100/100 [==============================] - 26s 259ms/step - loss: 0.6976 - accuracy: 0.4985 - val_loss: 0.6934 - val_accuracy: 0.6050
Epoch 2/20
100/100 [==============================] - 26s 257ms/step - loss: 0.6937 - accuracy: 0.5335 - val_loss: 0.6881 - val_accuracy: 0.5075
Epoch 3/20
100/100 [==============================] - 26s 260ms/step - loss: 0.6873 - accuracy: 0.5715 - val_loss: 0.6770 - val_accuracy: 0.6212
Epoch 4/20
100/100 [==============================] - 29s 292ms/step - loss: 0.6651 - accuracy: 0.6235 - val_loss: 0.6674 - val_accuracy: 0.6400
Epoch 5/20
100/100 [==============================] - 31s 315ms/step - loss: 0.6296 - accuracy: 0.6495 - val_loss: 0.6661 - val_accuracy: 0.6800
Epoch 6/20
100/100 [==============================] - 30s 295ms/step - loss: 0.6045 - accuracy: 0.6645 - val_loss: 0.6406 - val_accuracy: 0.6850
Epoch 7/20
100/100 [==============================] - 34s 340ms/step - loss: 0.5689 - accuracy: 0.7085 - val_loss: 0.5743 - val_ac